In [1]:
!open .

In [2]:
import pandas as pd
import NeoQuant as nq
import requests
import math

## get all company list

In [3]:
companies = nq.get_all_company_list()
companies.index = 'A' + companies.index

## bond spread

In [4]:
# https://www.kisrating.co.kr/ratingsStatistics/statics_spread.do
expected_ratio = 7.91

## get stock count

In [5]:
# company_name_list = ['이리츠코크렙', 'DL이앤씨', '맵스리얼티1']
company_name_list = ['신세계']

company_code_list = []
for company_name in company_name_list:
    company_code = nq.get_company_code(company_name, companies)
    if len(company_code) > 0:
        company_code_list.append(company_code)
    else:
        print('no company code with ' + company_name)
company_code_list

['A004170']

In [6]:
company_code_list = companies.index

## get fnguide info

In [7]:
import time

total = len(company_code_list)

for num, company_code in enumerate(company_code_list):
    try:
        company_name = nq.get_company_name(company_code, companies)
#         print(num, company_code, company_name)
        print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(company_code) + ')')

        time.sleep(1)

        try:
            snapshot_tables = nq.request_fnguide_snapshot(company_code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            snapshot_tables = fs_df = nq.request_fnguide_snapshot(company_code)

        if len(snapshot_tables) < 10:
            print('>>>> not much data')
            continue
        temp_df = nq.make_basic_df(company_code, snapshot_tables)
        temp_df['name'] = company_name
        if num == 0:
            basic_df = temp_df
        else:
            basic_df = pd.concat([basic_df, temp_df], sort=False)

        temp_df = nq.make_fr_df(company_code, snapshot_tables)
        if num == 0:
            fr_df = temp_df
        else:
            fr_df = pd.concat([fr_df, temp_df], sort=False)
    except ValueError:
        print('>>>> ValueError')
        continue
    except TypeError:
        print('>>>> TypeError')
        continue
    except KeyError:
        print('>>>> KeyError')
        continue
    
        
    if num == 20:
        break


1 / 2281 : 동화약품 (A000020)
2 / 2281 : KR모터스 (A000040)
3 / 2281 : 경방 (A000050)
4 / 2281 : 메리츠화재 (A000060)
5 / 2281 : 삼양홀딩스 (A000070)
6 / 2281 : 하이트진로 (A000080)
7 / 2281 : 유한양행 (A000100)
8 / 2281 : CJ대한통운 (A000120)
9 / 2281 : 하이트진로홀딩스 (A000140)
10 / 2281 : 두산 (A000150)
11 / 2281 : 성창기업지주 (A000180)


KeyboardInterrupt: 

## save basic dataframe to excel

In [ ]:
basic_df.to_excel(r'data/basic_data.xlsx')

## load basic dataframe from excel

In [8]:
basic_df = pd.read_excel(r'data/basic_data.xlsx')
basic_df.index = basic_df[basic_df.columns[0]]
basic_df.index.name = ''
basic_df.drop(basic_df.columns[0], axis = 1, inplace = True)

## save fr dataframe to excel

In [ ]:
fr_df.to_excel(r'data/fr_data.xlsx')

## load fr dataframe from excel

In [9]:
fr_df = pd.read_excel(r'data/fr_data.xlsx')
fr_df.index = fr_df[fr_df.columns[0]]
fr_df.index.name = ''
fr_df.drop(fr_df.columns[0], axis = 1, inplace = True)
big_col = list(fr_df.columns)
small_col = list(fr_df.iloc[0])

new_big_col = []
for num, col in enumerate(big_col):
    if 'Unnamed' in col:
        new_big_col.append(new_big_col[num-1])
    else:
        new_big_col.append(big_col[num])

fr_df.columns = [new_big_col, small_col]
fr_df = fr_df.loc[ fr_df.index.dropna() ]

## spread

In [10]:
expected_ratio = 7.91

## calculate adequate price

In [ ]:
total = len(fr_df)

for num, code in enumerate(fr_df.index):
    company_name = nq.get_company_name(code, companies)
    print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(code) + ')')

#     print(num, code, company_name)
    for col in reversed(fr_df.columns.levels[0]):
#         print(fr_df[col]['ROE'])
        try:
            roe = pd.to_numeric(fr_df.loc[code][col]['ROE'])
            per = pd.to_numeric(fr_df.loc[code][col]['PER'])
            pbr = pd.to_numeric(fr_df.loc[code][col]['PBR'])
            asset = pd.to_numeric(fr_df.loc[code][col]['지배주주지분']) * 100000000
            debt_rate = pd.to_numeric(fr_df.loc[code][col]['부채비율'])
            keep_rate = pd.to_numeric(fr_df.loc[code][col]['유보율'])
            interest_rate = pd.to_numeric(fr_df.loc[code][col]['배당수익률'])

                
            if not math.isnan(roe) and not math.isnan(asset):
                print('standard date : ' + col)
                stock_count = basic_df.loc[code]['주식수']
                basic_df.at[code, '기준일'] = col
                basic_df.at[code, 'roe'] = roe
                basic_df.at[code, 'per'] = per
                basic_df.at[code, 'pbr'] = pbr
                basic_df.at[code, '부채비율'] = debt_rate
                basic_df.at[code, '유보율'] = keep_rate
                basic_df.at[code, '배당수익률'] = interest_rate
                
                basic_df.at[code, 'expectation'] = expected_ratio
    #             print('stock_count : ' + str(stock_count))
                high = nq.get_more_adequate_price(asset, roe, expected_ratio, stock_count, 1)
    #             print('high : ' + str(high))
                basic_df.at[code, 'high'] = high
                middle = nq.get_more_adequate_price(asset, roe, expected_ratio, stock_count, 0.9)
                basic_df.at[code, 'middle'] = middle
                low = nq.get_more_adequate_price(asset, roe, expected_ratio, stock_count, 0.8)
                basic_df.at[code, 'low'] = low
                break
        except ValueError:
            print('>>> ValueError')
            continue


basic_df['dis_rate'] = basic_df['price'] / basic_df['low']
basic_df = basic_df.sort_values(by='dis_rate', ascending=True)

## make report

In [ ]:
final_df = basic_df
final_df = final_df.loc[~final_df.index.str.startswith('A9')]
final_df['dis_rate'] = final_df['price'] / final_df['low']
final_df = final_df.sort_values(by='dis_rate', ascending=True)

In [ ]:
final_df['rank'] = final_df['dis_rate'].rank()

In [ ]:
final_df = final_df.loc[final_df['dis_rate'] > 0.1]
final_df = final_df.loc[final_df['dis_rate'] < 1]
final_df = final_df.loc[final_df['roe'] > final_df['expectation']]

In [ ]:
final_df = final_df.loc[final_df['roe'] > 10]
final_df = final_df.loc[final_df['roe'] < 100]
final_df = final_df.loc[final_df['시가총액'] > 1000 * 100000000]

In [ ]:
final_df = final_df.loc[final_df['배당수익률'] > 1]

In [ ]:
final_df = final_df.loc[~final_df['name'].str.contains('홀딩스')]

In [ ]:
final_df = final_df.loc[final_df['roe'] > 10]

In [ ]:
len(final_df)

## save port

In [ ]:
final_df.to_excel(r'data/port_data.xlsx')

## load port

In [ ]:
# port_df = pd.read_excel(r'data/port_data.xlsx')
port_df = pd.read_excel(r'data/test_port_1.xlsx')
port_df.index = port_df[port_df.columns[0]]
port_df.index.name = ''
port_df.drop(port_df.columns[0], axis = 1, inplace = True)

## get recent price and update

In [ ]:
import time

port_list = port_df.index
total = len(port_list)
for num, company_code in enumerate(port_list):
    company_name = get_company_name(company_code, companies)
    print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(company_code) + ')')
    time.sleep(1)
    try:
        price_df = nq.request_price_list(company_code, 'day', 100)
    except requests.exceptions.Timeout:
        time.sleep(60)
        price_df = nq.request_price_list(company_code, 'day', 100)
        
    current = price_df[price_df.columns[0]][-1]
    port_df.at[company_code, 'current'] = int(current)


## 수익률 업데이트

In [ ]:
profit_rate = (port_df['current'] - port_df['price']) / port_df['price']
profit_rate = profit_rate.dropna()
profit_rate = profit_rate.map(lambda rate: str("%.2f" % round(float(rate * 100), 2)) + '%')
port_df['수익률'] = profit_rate

## 평균 수익률

In [ ]:
profit_rate = port_df['수익률'].map(lambda rate: float(rate[:-1]))
average_profit = profit_rate.sum() / len(port_df)
print('average profit : ' + "%.2f" % average_profit + '%')

## show chart

In [ ]:
company_name = '대원산업'
company_code = nq.get_company_code(company_name, companies)


price_df = nq.request_price_list(company_code, 'day', 1000)

price_high = basic_df.loc[company_code]['high']
price_middle = basic_df.loc[company_code]['middle']
price_low = basic_df.loc[company_code]['low']

price_df['price_low'] = [price_low] * len(price_df)
price_df['price_middle'] = [price_middle] * len(price_df)
price_df['price_high'] = [price_high] * len(price_df)

nq.show_price_chart(company_code, company_name, price_df)